In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('extremism_data_final.csv')

texts = df["Original_Message"].tolist()
labels = df["Extremism_Label"].tolist()


In [3]:
df['Extremism_Label'].value_counts()

Extremism_Label
NON_EXTREMIST    1987
EXTREMIST        1908
Name: count, dtype: int64

In [4]:
from sentence_transformers import SentenceTransformer
from sklearn.preprocessing import LabelEncoder


#model = SentenceTransformer("all-mpnet-base-v2")

model = SentenceTransformer("all-MiniLM-L6-v2")

le = LabelEncoder()
y = le.fit_transform(labels) 


X = model.encode(texts)


c:\Users\mahis\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
from sklearn.model_selection import train_test_split

# X = embeddings, y = labels numériques
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)


In [6]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


clf = LogisticRegression(max_iter=2000, class_weight="balanced")

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)


print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred, target_names=le.classes_))


Accuracy: 0.7278562259306803
               precision    recall  f1-score   support

    EXTREMIST       0.73      0.71      0.72       382
NON_EXTREMIST       0.73      0.75      0.74       397

     accuracy                           0.73       779
    macro avg       0.73      0.73      0.73       779
 weighted avg       0.73      0.73      0.73       779



In [7]:
X_new = model.encode(["degenrates like you belong to a cross"])  
y_new_pred = clf.predict(X_new)
pred_label = le.inverse_transform(y_new_pred)
print("Predicted label:", pred_label[0])


Predicted label: EXTREMIST


In [8]:
model.save("extremist_model", safe_serialization=False)


import joblib

joblib.dump(clf, "clf.pkl")
joblib.dump(le, "label_encoder.pkl")

['label_encoder.pkl']